In [ ]:
#query for efii2 deficit for weightloss using updated code for efi2 deficits
 CREATE OR REPLACE TABLE `yhcr-prd-phm-bia-core.CB_2172.cb_2172_efi2_wl` as (
#bring all snoemd and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  ctv3code,
  snomedcode,
  dateevent,
  cast(numericvalue as bignumeric) as must
  from `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    j.first_episodestartdate
    from a
    left join `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic` j on
    a.person_id = j.person_id),
#filter for Weight Loss snomed codes without timeconstraint
  b as (
    select
    person_id,
    snomedcode,
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Weight loss' AND TimeConstraintyears IS NULL AND Otherinstructions IS NULL) AND dateevent < first_episodestartdate),
    #snomedc code for must score greater than or equal to 1
  b_num as (
    select
    person_id,
    snomedcode,
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Weight loss' AND TimeConstraintyears IS NULL AND Otherinstructions IS NOT NULL) AND dateevent < first_episodestartdate and must >= 1),
#ctv3code with time constraint
  c as (
    select
    person_id,
    snomedcode, 
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Weight Loss' AND TimeConstraintyears IS NOT NULL) AND dateevent < first_episodestartdate AND datetime_diff(first_episodestartdate,dateevent,year) <= 5),
    #ctv3code without time constraint
    d as (
    select
    person_id,
    ctv3code,
    dateevent
    from a_master
    where ctv3code in(select CTV3 from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Weight Loss' AND TimeConstraintyears IS NULL) AND dateevent < first_episodestartdate),
#filter for weight loss csnomedcode with time constraint
  e as (
    select
    person_id,
    ctv3code, 
    dateevent
    from a_master
    where ctv3code in(select CTV3 from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Weight Loss' AND TimeConstraintyears IS NOT NULL) AND dateevent < first_episodestartdate AND datetime_diff(first_episodestartdate,dateevent,year) <= 5),
#union distinct individuals identified from ctv3 and snomed ct codes together
  snomed_ctv3 as (
    select
    * from b

    union distinct

    select *
    from c
    
    union distinct
    
    select *
    from d
    
    union distinct 
    
    select * 
    from e
    
    union distinct 
    
    select *
    from b_num)
#apply dummy variable
    select
    distinct person_id,
    case when dateevent IS NOT NULL then 'yes'
    end as weight_loss
    from snomed_ctv3)
  



